##### Importando as Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re
import warnings

##### Criando alguns filtros de Warning

In [2]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

##### Importando o Driver do Navegador

In [3]:
edge_path = 'edgedriver_win64/msedgedriver.exe'

##### Coletando dados do Campeonato Brasileiro em diferentes séries e anos

In [4]:
driver = webdriver.Edge(executable_path=edge_path)

series = ['a', 'b']

anos = range(2012, 2024)

dfs = []

for serie in series:
    for ano in anos:
        url = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-{serie}/{ano}'

        driver.get(url)

        time.sleep(2)

        table = driver.find_element(By.CSS_SELECTOR, 'table.table.m-b-20.tabela-expandir')

        df = pd.read_html(table.get_attribute('outerHTML'))[0]

        df['Ano'] = ano
        df['Serie'] = serie.upper()

        dfs.append(df)
driver.quit()

Esse código é utilizado para coletar dados de tabelas do site do campeonato brasileiro de futebol. Ele percorre as séries e anos definidos, acessa as páginas web correspondentes, aguarda 2 segundos para garantir que a página esteja completamente carregada, encontra a tabela de dados na página, lê o conteúdo da tabela, adiciona informações como o ano e a série aos dados coletados e armazena tudo em dois Dataframes(Série A e Série B).

##### Juntando os Dois DataFrames

In [5]:
df = pd.concat(dfs, ignore_index=True)
print(df)

                        Posição PTS   J   V   E   D  GP  GC  SG   CA  CV   %  \
0            1º Fluminense - RJ  77  38  22  11   5  61  33  28   85   3  67   
1              2º Atlético - MG  72  38  20  12   6  64  37  27  104  10  63   
2                3º Grêmio - RS  71  38  20  11   7  56  33  23  102   9  62   
3             4º São Paulo - SP  66  38  20   6  12  59  37  22   93   6  57   
4         5º Vasco da Gama - RJ  58  38  16  10  12  45  44   1   91   3  50   
..                          ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   
481  16º -3 Sampaio Corrêa - MA   5   6   1   2   3   6  10  -4   21   2  27   
482     17º -3 Ponte Preta - SP   5   6   1   2   3   4   9  -5   20   4  27   
483        18º -3 Tombense - MG   4   6   1   1   4   6   9  -3   14   1  22   
484       19º -1 Juventude - RS   3   6   1   0   5   3   7  -4   21   1  16   
485              20º 0 Abc - RN   1   6   0   1   5   3  12  -9   14   1   5   

      Ano Serie Recentes Próx  
0    20

##### Ajustando o Dataframe

In [6]:
df = df[~df.astype(str).apply(lambda x: x.str.contains('Observações')).any(axis=1)]
print(df)

                        Posição PTS   J   V   E   D  GP  GC  SG   CA  CV   %  \
0            1º Fluminense - RJ  77  38  22  11   5  61  33  28   85   3  67   
1              2º Atlético - MG  72  38  20  12   6  64  37  27  104  10  63   
2                3º Grêmio - RS  71  38  20  11   7  56  33  23  102   9  62   
3             4º São Paulo - SP  66  38  20   6  12  59  37  22   93   6  57   
4         5º Vasco da Gama - RJ  58  38  16  10  12  45  44   1   91   3  50   
..                          ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   
481  16º -3 Sampaio Corrêa - MA   5   6   1   2   3   6  10  -4   21   2  27   
482     17º -3 Ponte Preta - SP   5   6   1   2   3   4   9  -5   20   4  27   
483        18º -3 Tombense - MG   4   6   1   1   4   6   9  -3   14   1  22   
484       19º -1 Juventude - RS   3   6   1   0   5   3   7  -4   21   1  16   
485              20º 0 Abc - RN   1   6   0   1   5   3  12  -9   14   1   5   

      Ano Serie Recentes Próx  
0    20

In [7]:
df[['Posição', 'Time']] = df['Posição'].str.split(' ', 1, expand=True)
print(df)

    Posição PTS   J   V   E   D  GP  GC  SG   CA  CV   %   Ano Serie Recentes  \
0        1º  77  38  22  11   5  61  33  28   85   3  67  2012     A      NaN   
1        2º  72  38  20  12   6  64  37  27  104  10  63  2012     A      NaN   
2        3º  71  38  20  11   7  56  33  23  102   9  62  2012     A      NaN   
3        4º  66  38  20   6  12  59  37  22   93   6  57  2012     A      NaN   
4        5º  58  38  16  10  12  45  44   1   91   3  50  2012     A      NaN   
..      ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   ...   ...      ...   
481     16º   5   6   1   2   3   6  10  -4   21   2  27  2023     B    E V D   
482     17º   5   6   1   2   3   4   9  -5   20   4  27  2023     B    V E D   
483     18º   4   6   1   1   4   6   9  -3   14   1  22  2023     B    D D D   
484     19º   3   6   1   0   5   3   7  -4   21   1  16  2023     B    V D D   
485     20º   1   6   0   1   5   3  12  -9   14   1   5  2023     B    D E D   

    Próx                   

* o DataFrame tinha algumas linhas com Observações, então retiramos essas linhas.
* a Coluna de Posição guardava também o nome dos Times, então separamos em duas colunas.

Remoção de Números e Limpeza de Espaços em Branco na Coluna "Time"

In [8]:
def remove_numeros(texto):
    return re.sub(r'^[-+]?[0-9]+\.?[0-9]*', '', texto).strip()
df['Time'] = df['Time'].apply(remove_numeros)

In [9]:
print(df)

    Posição PTS   J   V   E   D  GP  GC  SG   CA  CV   %   Ano Serie Recentes  \
0        1º  77  38  22  11   5  61  33  28   85   3  67  2012     A      NaN   
1        2º  72  38  20  12   6  64  37  27  104  10  63  2012     A      NaN   
2        3º  71  38  20  11   7  56  33  23  102   9  62  2012     A      NaN   
3        4º  66  38  20   6  12  59  37  22   93   6  57  2012     A      NaN   
4        5º  58  38  16  10  12  45  44   1   91   3  50  2012     A      NaN   
..      ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   ...   ...      ...   
481     16º   5   6   1   2   3   6  10  -4   21   2  27  2023     B    E V D   
482     17º   5   6   1   2   3   4   9  -5   20   4  27  2023     B    V E D   
483     18º   4   6   1   1   4   6   9  -3   14   1  22  2023     B    D D D   
484     19º   3   6   1   0   5   3   7  -4   21   1  16  2023     B    V D D   
485     20º   1   6   0   1   5   3  12  -9   14   1   5  2023     B    D E D   

    Próx                 Ti

##### Excluindo Colunas Inúteis 

In [10]:
df=df.drop(['Recentes', 'Próx'], axis=1)
print(df)

    Posição PTS   J   V   E   D  GP  GC  SG   CA  CV   %   Ano Serie  \
0        1º  77  38  22  11   5  61  33  28   85   3  67  2012     A   
1        2º  72  38  20  12   6  64  37  27  104  10  63  2012     A   
2        3º  71  38  20  11   7  56  33  23  102   9  62  2012     A   
3        4º  66  38  20   6  12  59  37  22   93   6  57  2012     A   
4        5º  58  38  16  10  12  45  44   1   91   3  50  2012     A   
..      ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   ...   ...   
481     16º   5   6   1   2   3   6  10  -4   21   2  27  2023     B   
482     17º   5   6   1   2   3   4   9  -5   20   4  27  2023     B   
483     18º   4   6   1   1   4   6   9  -3   14   1  22  2023     B   
484     19º   3   6   1   0   5   3   7  -4   21   1  16  2023     B   
485     20º   1   6   0   1   5   3  12  -9   14   1   5  2023     B   

                    Time  
0        Fluminense - RJ  
1          Atlético - MG  
2            Grêmio - RS  
3         São Paulo - SP  


##### Setando o DataFrame

In [11]:
df["Time"] = df["Time"].replace('Atlético Paranaense - PR', 'Athletico Paranaense - PR')
df["Time"] = df["Time"].replace('Atlético - MG', 'Atlético Mineiro - MG')
df["Time"] = df["Time"].replace('Bragantino - SP', 'Red Bull Bragantino - SP')

##### Extraindo o Estado de Cada Time

In [12]:
df['Estado'] = df['Time'].str.extract(r'-(\s*[A-Z]{2})$')
df['Time'] = df['Time'].str.replace(r'\s*-\s*[A-Z]{2}$', '', regex=True)
df['Time'] = df['Time'].str.replace(' Saf$', '')
df['Time'] = df['Time'].str.replace(' S.a.f.$', '')
df['Time'] = df['Time'].str.strip()

In [13]:
print(df)

    Posição PTS   J   V   E   D  GP  GC  SG   CA  CV   %   Ano Serie  \
0        1º  77  38  22  11   5  61  33  28   85   3  67  2012     A   
1        2º  72  38  20  12   6  64  37  27  104  10  63  2012     A   
2        3º  71  38  20  11   7  56  33  23  102   9  62  2012     A   
3        4º  66  38  20   6  12  59  37  22   93   6  57  2012     A   
4        5º  58  38  16  10  12  45  44   1   91   3  50  2012     A   
..      ...  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   ...   ...   
481     16º   5   6   1   2   3   6  10  -4   21   2  27  2023     B   
482     17º   5   6   1   2   3   4   9  -5   20   4  27  2023     B   
483     18º   4   6   1   1   4   6   9  -3   14   1  22  2023     B   
484     19º   3   6   1   0   5   3   7  -4   21   1  16  2023     B   
485     20º   1   6   0   1   5   3  12  -9   14   1   5  2023     B   

                 Time Estado  
0          Fluminense     RJ  
1    Atlético Mineiro     MG  
2              Grêmio     RS  
3          

##### Ordenando o DataFrame por ano e série

In [14]:
df.sort_values(by=['Ano', 'Serie'], inplace=True)

##### Criando uma coluna para indicar em qual série o time estava no ano anterior

In [15]:
df['Serie_Anterior'] = df.groupby('Time')['Serie'].shift(1)

##### Criando uma lista com todos os anos em que há dados

In [16]:
anos = df['Ano'].unique()

##### Verificando se um time aparece no ano anterior

In [17]:
for i, ano in enumerate(anos):
    if i == 0:
        continue 
    ano_anterior = anos[i - 1]
    for time in df['Time'].unique():
        if (time in df[df['Ano'] == ano_anterior]['Time'].unique()):
            df.loc[(df['Ano'] == ano) & (df['Time'] == time), 'Serie_Anterior'] = \
            df.loc[(df['Ano'] == ano_anterior) & (df['Time'] == time), 'Serie'].values[0]
        else:
            df.loc[(df['Ano'] == ano) & (df['Time'] == time), 'Serie_Anterior'] = 'C'

        if (df.loc[(df['Ano'] == ano) & (df['Time'] == time), 'Serie'] == 'A').all() and \
        (df.loc[(df['Ano'] == ano) & (df['Time'] == time), 'Serie_Anterior'] == 'C').all():
            df.loc[(df['Ano'] == ano) & (df['Time'] == time), 'Serie_Anterior'] = 'B'

Esse codigo primeiro pula primeiro Ano e logo após começa a verificar se o time estava na Serie A, B ou C no Ano Anterior. 
Fiz isso para criar uma variável Binária de Pesos baseado em qual Série o time estava no Ano Anterior.

##### Definindo os pesos de acordo com a série do ano anterior

In [18]:
df.loc[df['Serie_Anterior'] == 'A', 'Peso'] = 1.0
df.loc[df['Serie_Anterior'] == 'B', 'Peso'] = 0.7
df.loc[df['Serie_Anterior'] == 'C', 'Peso'] = 0.5

##### Removendo o Primeiro Ano do Dataset

In [19]:
df = df[df['Ano'] != 2012]

Removi os Dados Relativos ao Ano de 2012 pelo fato deles estarem Desbalanceados

##### Setando a Ordem das Colunas

In [20]:
df=df.reindex(columns=['Posição', 'Time', 'PTS','Estado', 'J', 'V', 'E', 'D', 'GP', 'GC', 'SG', 'CA', 'CV', '%', 'Ano','Serie','Serie_Anterior','Peso'])

In [21]:
print(df)

    Posição                  Time PTS Estado   J   V   E   D  GP  GC  SG  CA  \
20       1º              Cruzeiro  76     MG  38  23   7   8  77  37  40  73   
21       2º                Grêmio  65     RS  38  18  11   9  42  35   7  79   
22       3º  Athletico Paranaense  64     PR  38  18  10  10  65  49  16  92   
23       4º              Botafogo  61     RJ  38  17  10  11  55  41  14  82   
24       5º               Vitória  59     BA  38  16  11  11  59  53   6  67   
..      ...                   ...  ..    ...  ..  ..  ..  ..  ..  ..  ..  ..   
481     16º        Sampaio Corrêa   5     MA   6   1   2   3   6  10  -4  21   
482     17º           Ponte Preta   5     SP   6   1   2   3   4   9  -5  20   
483     18º              Tombense   4     MG   6   1   1   4   6   9  -3  14   
484     19º             Juventude   3     RS   6   1   0   5   3   7  -4  21   
485     20º                   Abc   1     RN   6   0   1   5   3  12  -9  14   

    CV   %   Ano Serie Serie_Anterior  

##### Criando um CSV com os Dados obtidos

In [22]:
df.to_csv('brasileirao.csv', index=False)